In [207]:
import requests



In [208]:
path = "/healthz"
base_url = "http://localhost:8002" 
endpoint= f"{base_url}{path}"
endpoint
 



'http://localhost:8002/healthz'

In [209]:
response = requests.get(endpoint)
print(response.ok )
print(response.text)


True
{"status":"healthy"}


In [219]:
path = "/api/events"
base_url = "http://localhost:8002" 
endpoint= f"{base_url}{path}"
endpoint

response = requests.get(endpoint)
if response.ok:
    data = response.json()
    print(data)
    print(response.text)


else:
    print(response.text)
    print(response.status_code)

{'result': [{'create_at': '2026-01-13T08:13:09.375109Z', 'page': '/test', 'description': '', 'id': 1}], 'count': 1}
{"result":[{"create_at":"2026-01-13T08:13:09.375109Z","page":"/test","description":"","id":1}],"count":1}


In [232]:
path = "/api/events/1"
base_url = "http://localhost:8002" 
endpoint= f"{base_url}{path}"
endpoint


response = requests.get(endpoint)
if response.ok:
    data = response.json()
    print(response.headers )
    print(data, type(data))
    print(response.text)
    


else:
    print(response.status_code)
    print(response.text)
    print(endpoint)


500
Internal Server Error
http://localhost:8002/api/events/1


In [59]:
import json


In [250]:
#POST create event 

data = {"page":"/test"}


path = "/api/events/"
base_url = "http://localhost:8002" 
create_endpoint= f"{base_url}{path}"
create_endpoint


response = requests.post(create_endpoint, json=data)

if response.ok:
    data = response.json()
    print(response.ok)
    print(response.headers )
    print(data, type(data))
    
    
else:
    print(response.status_code)
    print(response.ok)
    print(response.text)
    print(endpoint)




True
{'date': 'Tue, 13 Jan 2026 09:03:13 GMT', 'server': 'uvicorn', 'content-length': '124', 'content-type': 'application/json'}
{'page': '/test', 'create_at': '2026-01-13T09:03:13.808886Z', 'description': '', 'id': 4, 'update_at': '2026-01-13T09:03:13.808891Z'} <class 'dict'>


In [256]:
#GET all  events
from pprint import  pprint

path = "/api/events"
base_url = "http://localhost:8002" 
detail_endpoint= f"{base_url}{path}"
endpoint


response = requests.get(detail_endpoint)

if response.ok:
    data = response.json()
    print(response.ok)
    print(response.headers )
    pprint(data)

    
    
else:
    print(response.status_code)
    print(response.ok)
    print(response.text)
    print(endpoint)



True
{'date': 'Tue, 13 Jan 2026 09:04:08 GMT', 'server': 'uvicorn', 'content-length': '160', 'content-type': 'application/json'}
{'count': 1,
 'result': [{'create_at': '2026-01-13T09:03:01.764852Z',
             'description': 'inline update',
             'id': 3,
             'page': '/test',
             'update_at': '2026-01-13T09:04:00.589330Z'}]}


In [255]:
#PUT update event
from pprint import  pprint

path = "/api/events/3"
base_url = "http://localhost:8002" 
detail_endpoint= f"{base_url}{path}"
endpoint


response = requests.put(detail_endpoint, json={"description":"inline update"})

if response.ok:
    data = response.json()
    print(response.ok)
    print(response.headers )
    pprint(data)

    
    
else:
    print(response.status_code)
    print(response.ok)
    print(response.text)
    print(endpoint)



True
{'date': 'Tue, 13 Jan 2026 09:04:00 GMT', 'server': 'uvicorn', 'content-length': '137', 'content-type': 'application/json'}
{'create_at': '2026-01-13T09:03:01.764852Z',
 'description': 'inline update',
 'id': 3,
 'page': '/test',
 'update_at': '2026-01-13T09:04:00.589330Z'}


In [252]:

from pprint import  pprint

path = "/api/events/4"
base_url = "http://localhost:8002" 
detail_endpoint= f"{base_url}{path}"
detail_endpoint


response = requests.delete(detail_endpoint)

    
    


